<a href="https://colab.research.google.com/github/deboradcm/IARTES/blob/master/04_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 07 - Naive Bayes

**Índice**

  - [Carregando o conjunto de dados](#Carregando-o-conjunto-de-dados)
  - [Naive Bayes "na mão"](#Naive-Bayes-"na-mão")
    - [Probabilidades a priori](#Probabilidades-a-priori)
    - [Cálculo das verossimilhanças "ingênuas"](#Cálculo-das-verossimilhanças-"ingênuas")
    - [Classificando um exemplo](#Classificando-um-exemplo)
  - [Uma classe para Naive Bayes](#Uma-classe-para-Naive-Bayes)
  - [Naive Bayes com o Scikit-Learn](#Naive-Bayes-com-o-Scikit-Learn)

## Carregando o conjunto de dados

Especifica o diretório onde os dados estão. Melhor usar em todos os *notebooks* para que você possa manter todos  arquivos de dados juntos.

Caso a localização dos dados no seu computador seja diferente, troque a variável abaixo. Caso esteja no mesmo diretório (*e.g.*, no Colaboratory), coloque `DIR_DADOS = ./`.

In [ ]:
DIR_DADOS = '../dados/'

Comandos iniciando com `!` são executados pelo *shell* do sistema operacional. Por exemplo, a imagem abaixo foi gerada quando o *notebook* foi executado pela última vez, mostrando o conteúdo do diretório `../dados`.

In [ ]:
!ls ../dados/

ls: cannot access '../dados/': No such file or directory


Vamos trabalhar com o conjunto de dados `playtennis`, que está disponibilizado no ColabWeb como um arquivo CSV.

O jeito mais fácil de trabalhar arquivos CSV em Python é utilizando o Pandas. Normalmente nós importamos o Pandas dando a ele o apelido `pd`.

In [ ]:
import pandas as pd

A função `pandas.read_csv` lê o arquivo como CSV em um objeto `pandas.DataFrame`.

In [ ]:
tennis = pd.read_csv(DIR_DADOS + 'tennis.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../dados/tennis.csv'

In [ ]:
type(tennis)

O *data frame* é uma estrutura de dados que se comporta como uma matriz que contém dois índices.

In [ ]:
tennis

Eis o índice das colunas...

In [ ]:
tennis.columns

E o índice das linhas:

In [ ]:
tennis.index

Se você utilizar o operador sobrecarregado `[]`, você pode acessar uma coluna por meio de sua chave no índice. Por exemplo:

In [ ]:
tennis['aparencia']

Quando você acessa uma coluna ou uma linha em um *data frame*, o resultado é um objeto da classe `Series`, também do Pandas.

Se o *data frame* é uma matriz com dois índices, então a série é um vetor com um único índice. No caso do acesso às colunas, o índice da série é o índice das linhas do *data frame*.

In [ ]:
type(tennis['aparencia'])

Além do acesso implícito por meio do operador sobrecarregado `[]`, o `DataFrame` permite acecssar de maneira explícita o índice `iloc`, que é puramente posicional, e o índice `loc`, que usa chaves para encontrar as linhas e as colunas.

In [ ]:
tennis.loc[3, 'aparencia']

In [ ]:
tennis.iloc[0, 1]

Para vermos os valores únicos, podemos usar o método `drop_duplicates()` do Series.

In [ ]:
tennis['aparencia'].drop_duplicates()

In [ ]:
tennis['aparencia'].value_counts()

## Naive Bayes "na mão"

Vamos implementar primeiro o Naive Bayes. Vamos praticar um pouco manipulando o *data frame*. Depois vamos escrever uma classse que faz o treinamento, isto é, que aprende os parâmetros para generalizar os dados passados.

### Probabilidades a priori

Vamos começar calculando as probabilidades priori. Neste momento, vamos fazer isso especificamente para o problema `playtennis`, fazendo uso do conhecimento que existem duas classes.

Para calcular as prioris, fazemos a contagem dos exemplos do conjunto de treinamento para estimar as frequências de ambas as classes.

Primeiro, separamos os exemplos da classe positiva dos exemplos da classe negativa.

In [ ]:
idx_sim = tennis['jogar'] == 'sim'

Essa comparação vai produzir uma máscara, um índice de onde se encontram os exemplos da classe positiva.

In [ ]:
idx_sim

Utilizando o operador de negação bit-a-bit, podemos gerar a máscara dos exemplos negativos:

In [ ]:
idx_nao = ~idx_sim

In [ ]:
idx_nao

Agora, aplicando essa máscara no *data frame*, podemos obter apenas as linhas que contém exemplos positivos ou negativos.

In [ ]:
exemplos_sim = tennis[idx_sim]

In [ ]:
exemplos_sim

As primeiras frequências que precisamos guardar é, portanto, a frequência das duas classes. Ou seja, as probabilidades a priori.

In [ ]:
exemplos_nao = tennis[idx_nao]

In [ ]:
exemplos_nao

Excelente.

O próximo passo é calcular os números de exemplos. As classes do Pandas (e também as do NumPy) possuem um atributo chamado `shape` que armazena os tamanhos das dimensões dos objetos.

No caso de *data frames*, a propriedade `pandas.DataFrame.shape` é uma tupla com dois valores. O primeiro é o número de linhas e o segundo é o número de colunas. Usaremos essa informação para calcular o número de exemplos de cada classe.

In [ ]:
tennis.shape

In [ ]:
num_sim = exemplos_sim.shape[0]
num_sim

In [ ]:
num_nao = exemplos_nao.shape[0]
num_nao

Vamos criar um dicionário chamado `nb_tennis` no qual iremos guardar todas as probabilidades (prioris e verossimilhanças).

In [ ]:
nb_tennis = {}

In [ ]:
nb_tennis['sim'] = num_sim / (num_sim + num_nao)

In [ ]:
nb_tennis['nao'] = num_nao / (num_sim + num_nao)

In [ ]:
nb_tennis

### Adendo: *list comprehensions*, *lambda*

O restante dos cálculos poderia ser feito de duas formas. A primeira é com "programação tradicional", usando `for`, iteradores e acumuladores. Mas podemos utilizar recursos de programação funcional do Python para fazer a mesma coisa mais rapidamente.

O primeiro recurso que vamos utilizar aqui é o de função lambda. A função lambda é uma função anônima que pode ser declarada como uma expressão.

Uma declaração "tradicional" de função é feita com `def`. Nome a sintaxe:

    def IDENTIFICADOR(PARAMETROS):
        CORPO

Quando essa declaração é executada, um nome é definido e associado à função.

In [ ]:
def fsoma(x, y):
    return x + y

In [ ]:
type(fsoma)

Se utilizarmos esse nome em uma chamada, a função é executada e o valor retornado é avaliado como a saída da célula.

In [ ]:
fsoma(4, 2)

A mesma função poderia ser definida com expressão lambda. A expressão lambda cria uma função anônima, que podemos referenciar com uma variável.

A sintaxe é bem parecida. Compare:

```Python
def    fsoma(x, y): return x + y
lambda       x, y :        x + y
```
      
Essencialmente, a palavra `def` é substituída pela palavra `lambda` e, como a função lambda é anônima, o identificador da função some. Os parênteses na lista de parâmetros são desnecessários. E, como tudo o que uma função lambda pode fazer é executar um único comando `return`, a palavra `return` também é desnecessária.

Para não perdermos acesso à função anônima, iremos referenciá-la com uma variável.

In [ ]:
lsoma = lambda x, y: x + y

Essa variável pode ser utilizada do mesmo jeito que o identificador de funções é utilizado:

In [ ]:
lsoma(4, 2)

O segundo recurso que vamos utilizar aqui é o de *list comprehension* ou compreensão de listas. A compreensão de lsitas permite declarar listas com uma notação semelhante à que utilizamos na matemática para descrever conjuntos implícitos.

Por exemplo, o conjunto de todos os números quadrados no intervalo $[0,10]$ pode ser descrito da seguinte forma:

$$\{x^2 | 0 \leq x \leq 10 \wedge x \in \mathbb{Z}\}$$

Em Python, uma lista com esses valores poderia ser escrita da seguinte maneira:

In [ ]:
[x**2 for x in range(11)]

A notação é:

```Python
[EXPRESSAO for VARIAVEL in CONJUNTO]
```

Nesse caso, `range(11)` sozinho já especifica ao mesmo tempo que todos os valores estão no intervalo $[0, 10]$ e também que são números inteiros.

Na compreensão de listas também podemos especificar uma condição. Por exemplo, se quisermos todos os números pares entre 0 e 10, a notação matemática é:

$$\{x | 0 \leq x \leq 100 \wedge x \in \mathbb{Z} \wedge x \equiv 0 (\text{mod } 2)\}$$

Em Python, as condições da compreensão de listas podem ser especificadas com um `if`:

In [ ]:
[x for x in range(11) if x % 2 == 0]

No restante do *notebook*, vamos ver os comandos com compreensão de listas.

### Cálculo das verossimilhanças "ingênuas"

Agora vamos calcular as verossimilhanças. Para isso, faremos uso do método `value_counts()`, que retorna uma contagem de cada valor de uma série.

Por exemplo, se quisermos saber quantas vezes cada valor de aparência aparece na coluna `aparencia`, podemos fazer o seguinte:

In [ ]:
tennis['aparencia'].value_counts()

Se fizermos isso apenas para os exemplos da classe positiva e depois dividirmos pelo número de exemplos dessa classe, o resultado será $p(\textsf{aparencia}=x_i|\textsf{sim})$ para cada valor $x_i$ do atributo `aparencia`:

In [ ]:
exemplos_sim['aparencia'].value_counts() / num_sim

Fascinante!

Então, vamos fazer a mesma coisa para todos os atributos. Primeiro, vamos declarar uma lista que contém todos os nomes dos atributos. Essa lista vai nos ajudar a montar a compreensão de lista depois.

In [ ]:
atributos = ['aparencia', 'temperatura', 'umidade', 'vento']
atributos

Veja que é bem semelhante ao exemplo anterior. Se tivermos uma compreensão `[EXPRESSÃO for VAR in atributos]`, então a lista será construída com base na expressão para cada valor da lista `atributos`.

Se a expressão for simplesmente o nome da variável, então o resultado será uma cópia da lista original:

In [ ]:
[X for X in atributos]

Mas podemos usar a variável em algumas expressões mais interessantes. Por exemplo, se fizermos `len(X)`, então o resultado será uma lista equivalente à seguinte:

```Python
[len(atributos[0]), len(atributos[1]), len(atributos[2]), len(atributos[3])]
```

In [ ]:
[len(X) for X in atributos]

Da mesma maneira, podemos usar `X` em uma expressão que obtém as verossimilhanças:

In [ ]:
freq_sim = [exemplos_sim[X].value_counts() / num_sim for X in atributos]
freq_sim

Esse código seria equivalente ao seguinte:

```Python
# Cria uma lista com o tamanho certo e valores iniciais quaiser
freq_sim = []
for X in atributos:
    freq_sim.append(exemplos_sim[X].value_counts() / num_sim)
```

Note como a compreensão de listas é be mais expressiva. Além disso, em muitas situações ela pode ser bem mais eficiente. Por exemplo, na compreensão de lista o tamanho da lista não muda com cada iteração do laço `for`.

Agora fazemos o mesmo para a verossimilhanças da classe negativa.

In [ ]:
freq_nao = [tennis[idx_nao][X].value_counts() / num_nao for X in atributos]
freq_nao

E finalmente guardamos o resultado no dicionário.

In [ ]:
nb_tennis['vsim'] = freq_sim
nb_tennis['vnao'] = freq_nao

In [ ]:
nb_tennis

### Classificando um exemplo

Agora vamos pensar no código que realiza a classificação. Para simplificar, vamos supor que o exemplo a ser classificado está numa série do Pandas.

Então vamos começar recriando o exemplo do dia 15 que vimos em aula.

In [ ]:
atributos = ['ensolarado', 'moderado', 'alta', 'forte']

In [ ]:
valores = ['aparencia', 'temperatura', 'umidade', 'vento']

In [ ]:
exemplo = pd.Series(atributos, valores)
exemplo

Para classificar esse exemplo, precisamos descobrir se $p(\textsf{sim}|x)$ é maior que $p(\textsf{não}|x)$ ou o contrário. Isso significa estimar os valores proporcionais a eles multiplicando as verossimilhanças pelas prioris.

No dicionário que criamos, `nb_tennis['vsim']` é uma lista que contém todas as verossimilhanças de cada exemplo. O primeiro elemento dessa lista é o conjunto das verossimilhanças $p(\textsf{aparencia}=x_i|\textsf{sim})$. Se indexarmos essa lista pelo valor que o exemplo possui para o atributo `aparencia`, o resultado será $p(\textsf{ensolarado}|\textsf{sim})$:

In [ ]:
nb_tennis['vsim'][0]['ensolarado']

Ou:

In [ ]:
nb_tennis['vsim'][0][exemplo[0]]

Se empregarmos isso em uma compreensão de lista, o resultado será o conjunto de todas as verossimilhanças que procuramos:

In [ ]:
[nb_tennis['vsim'][X][exemplo[X]] for X in range(4)]

Essa compreensão é equivalente à seguinte iteração:

```Python
l = []
for X in range(4):
    l.append(nb_tennis['vsim'][X][exemplo[X]])
```

Agora podemos fazer uso do NumPy para multiplicar todos esses valores. A função `numpy.prod()` recebe como entra uma coleção que pode ser um vetor, uma matriz, um *data frame* ou semelhante, e retorna o produto de todos os elementos dessa coleção.

Por exemplo, o produto de `2 * 3 * 4 * 5` é 120:

In [ ]:
import numpy as np

In [ ]:
np.prod([2, 3, 4, 5])

Usamos `np.prod` para multiplicar todas as verossimilhanças e depois multiplicamos pela priori de cada classe:

In [ ]:
np.prod([nb_tennis['vsim'][X][exemplo[X]] for X in range(4)]) * nb_tennis['sim']

Esse código é equivalente ao seguinte:

```Python
prod = 1
for X in range(4):
    prod *= nb_tennis['vsim'][X][exemplo[X]]
prod *= nb_tennis['sim']
prod
```

E fazemos o mesmo para a classe negativa:

In [ ]:
np.prod([nb_tennis['vnao'][X][exemplo[X]] for X in range(4)]) * nb_tennis['nao']

Como você pode ver, isso nos levaria à decisão de que a classe positiva é a mais provável, assim como concluímos com os exemplos dos slides.

## Uma classe para Naive Bayes

Agora podemos juntar tudo em uma classe. Ela tem o método `fit` para, assim como os modelos do Scikit, aprender os parâmetros do modelo para um conjunto de treinamento. E tem também o método `predict`, que faz inferência. Diferentemente do Scikit, nossa classe só faz inferência em um exemplo.

In [ ]:
class NaiveBayes:
    def __init__(self):
        self._prioris = {}
        self._likelihoods = {}

        self._labels = []
        self._num_labels = -1

        self._features = []
        self._num_features = -1

    def _estimate_frequencies(self, series):
        return series.value_counts() / series.shape[0]

    def fit(self, X, y):
        self._labels = list(y.drop_duplicates())
        self._features = X.keys().values

        self._num_features = X.shape[1]
        self._num_labels = len(self._labels)

        self._prioris = self._estimate_frequencies(y)
        for label in self._labels:
            subset = X[y == label]
            self._likelihoods[label] = [self._estimate_frequencies(subset[X]) for X in self._features]

    def _get_likelihood(self, feature, value, label):
        return self._likelihoods[label][feature][value]

    def predict(self, Xpred):
        ypred = None
        ypred_chance = -1

        chance_sum = 0
        prob = np.zeros((self._num_labels,), dtype=np.float64)

        for l in range(self._num_labels):
            label = self._labels[l]
            this_priori = self._prioris[label]
            this_chance = np.prod([self._get_likelihood(X, Xpred[X], label) for X in range(4)]) * this_priori

            prob[l] = this_chance
            chance_sum += this_chance

            if this_chance > ypred_chance:
                ypred = label
                ypred_chance = this_chance

        prob /= chance_sum
        return (ypred, prob)

Para treinar, primeiro separamos os dados em atributos e rótulos.

Para a nossa classe, é imperativo que os dados estejam em um `DataFrame`  e os rótulos estejam em um `Series`. O Scikit não é tão exigente, apenas requer matrizes para as características e veteores para os rótulos.

In [ ]:
X = tennis.iloc[:, :-1]
X

In [ ]:
y = tennis.iloc[:, -1]
y

Agora, assim como no Scikit, usamos o método `fit` para aprender os parâmetros.

In [ ]:
nb = NaiveBayes()
nb.fit(X, y)

Mostre os parâmetros aprendidos:

In [ ]:
nb._prioris

In [ ]:
nb._likelihoods

Faça predição do exemplo e veja o resultado e as probabilidades **estimadas**.

In [ ]:
exemplo

In [ ]:
nb.predict(exemplo)

## Naive Bayes com o Scikit-Learn

Agora, vamos utilizar o Naive Bayes categórico do ScikitLearn.

Antes de começar, vamos importar novamente os dados para que as manipulações da seção anterior não nos atrapalhem. Vamos separar as características em uma matriz X e os rótulos em um vetor y.

In [ ]:
import pandas as pd
import numpy as np

Carregue os dados.

In [ ]:
tennis = pd.read_csv("/content/tennis.csv");
tennis

,aparencia,temperatura,umidade,vento,jogar
0,ensolarado,quente,alta,fraco,nao
1,ensolarado,quente,alta,forte,nao
2,nublado,quente,alta,fraco,sim
3,chuvoso,moderado,alta,fraco,sim
4,chuvoso,frio,baixa,fraco,sim
5,chuvoso,frio,baixa,forte,nao
6,nublado,frio,baixa,forte,sim
7,ensolarado,moderado,alta,fraco,nao
8,ensolarado,frio,baixa,fraco,sim
9,chuvoso,moderado,baixa,fraco,sim


Separe os dados em `X` (data frame/matriz) e `y` (série/vetor).

In [ ]:
X = tennis.iloc[:,:-1] #[0:-1] pega todas as colunas até a última coluna deixando a última de fora [0; tam -1]
X

,aparencia,temperatura,umidade,vento
0,ensolarado,quente,alta,fraco
1,ensolarado,quente,alta,forte
2,nublado,quente,alta,fraco
3,chuvoso,moderado,alta,fraco
4,chuvoso,frio,baixa,fraco
5,chuvoso,frio,baixa,forte
6,nublado,frio,baixa,forte
7,ensolarado,moderado,alta,fraco
8,ensolarado,frio,baixa,fraco
9,chuvoso,moderado,baixa,fraco


In [ ]:
y = tennis.iloc[:,-1]
y

0     nao
1     nao
2     sim
3     sim
4     sim
5     nao
6     sim
7     nao
8     sim
9     sim
10    sim
11    sim
12    sim
13    nao
Name: jogar, dtype: object

In [ ]:
X, y

(     aparencia temperatura umidade  vento
 0   ensolarado      quente    alta  fraco
 1   ensolarado      quente    alta  forte
 2      nublado      quente    alta  fraco
 3      chuvoso    moderado    alta  fraco
 4      chuvoso        frio   baixa  fraco
 5      chuvoso        frio   baixa  forte
 6      nublado        frio   baixa  forte
 7   ensolarado    moderado    alta  fraco
 8   ensolarado        frio   baixa  fraco
 9      chuvoso    moderado   baixa  fraco
 10  ensolarado    moderado   baixa  forte
 11     nublado    moderado    alta  forte
 12     nublado      quente   baixa  fraco
 13     chuvoso    moderado    alta  forte,
 0     nao
 1     nao
 2     sim
 3     sim
 4     sim
 5     nao
 6     sim
 7     nao
 8     sim
 9     sim
 10    sim
 11    sim
 12    sim
 13    nao
 Name: jogar, dtype: object)

O scikit-learn implementa diferentes modelos de classificação baseados em probabilidades no módulo `sklearn.naive_bayes`. Eles se diferem pela forma como calculam as probabilidades dos atributos.

O Naive Bayes categórico é implementado pela classe `sklearn.naive_bayes.CategoricalNB`. Por exemplo, para estimar a verossimilhança, ele usa a seguinte fórmula,

$$p(x|y) = \frac{N_x+\alpha}{N_y+\alpha{}n_i},$$

na qual as variáveis representam o seguinte:

- $N_x$ é o número de exemplos da classe $y$ que possuem o valor $x$ para o atributo em questão;
- $N_y$ é o número de exemplos da classe $y$;
- $\alpha$ é um fator de ajuste;
- $n_i$ é o número de valores distintos que o atributo em questão pode assumir.

Tomando o conjunto `play_tennis` como exemplo, tínhamos o seguinte:

- Havia 9 instâncias da classe `sim` no conjunto de treinamento;
- O atributo `aparencia` possuía três possíveis valores: `ensolarado`, `nublado` e `chuvoso`;
- Das 9 instâncias da classe `sim`, duas possuem valor `ensolarado`.

Nesse caso, a verossimilhança do valor `ensolarado` seria calculada da seguinte maneira:

$$p(\textsf{ensolarado}|\textsf{sim})=\frac{2+\alpha}{9+3\alpha}$$

Isso se chama suavização de Laplace e serve para resolver problemas que podem acontecer quando um exemplo possui um valor que não estava presente no conjunto de treinamento.

Vamos começar importando esta classe.

In [ ]:
from sklearn.naive_bayes import CategoricalNB
import sklearn.naive_bayes
dir(sklearn.naive_bayes)

['ABCMeta',
 'BaseEstimator',
 'BernoulliNB',
 'CategoricalNB',
 'ClassifierMixin',
 'ComplementNB',
 'GaussianNB',
 'Hidden',
 'Integral',
 'Interval',
 'LabelBinarizer',
 'MultinomialNB',
 'Real',
 'StrOptions',
 '_BaseDiscreteNB',
 '_BaseNB',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_check_partial_fit_first_call',
 '_check_sample_weight',
 'abstractmethod',
 'binarize',
 'check_is_fitted',
 'check_non_negative',
 'label_binarize',
 'logsumexp',
 'np',
 'safe_sparse_dot',
 'warnings']

Como no problema `play_tennis` todas as categorias ocorrem para todos os exemplos pelo menos uma vez, não há possibilidade de um exemplo de inferência conter valores novos. Portanto podemos especificar um valor bem pequeno para esse hiperparâmetro—por exemplo, $\alpha=1\times10^{-10}$.

In [ ]:
nb = CategoricalNB(alpha=1e-10)

Entretanto, se tentarmos treinar o classificador Naive Bayes com o  conjunto de atributos que selecionamos, uma exceção será lançada:

In [ ]:
nb.fit(X, y)

ValueError: invalid literal for int() with base 10: 'ensolarado'

De modo geral, todos os indutores do Scikit-Learn exigem que os atributos sejam numéricos. Neste caso em que os atributos são categóricos, eles precisam ser mapeados para um conjunto de valores numéricos e discretos.

Isso pode ser feito, por exemplo, com a classe `OrdinalEncoder`.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

Vamos começar criando um codificador para as características armazenadas em `X`.

Instancie um objeto da calsse `OrdinalEncoder` e "treine-o" nos dados de entrada.

In [ ]:
tennis_oe = OrdinalEncoder()

In [ ]:
tennis_oe.fit(X);

Agora, aplique a transformação aos exemplos originais, produzindo uma nova versão da matriz `X`.

In [ ]:
Xenc = tennis_oe.transform(X)
Xenc

array([[1., 2., 0., 1.],
       [1., 2., 0., 0.],
       [2., 2., 0., 1.],
       [0., 1., 0., 1.],
       [0., 0., 1., 1.],
       [0., 0., 1., 0.],
       [2., 0., 1., 0.],
       [1., 1., 0., 1.],
       [1., 0., 1., 1.],
       [0., 1., 1., 1.],
       [1., 1., 1., 0.],
       [2., 1., 0., 0.],
       [2., 2., 1., 1.],
       [0., 1., 0., 0.]])

Precisamos fazer a mesma coisa para os rótulos. Entretanto, a classe `OrdinalEncoder` só trabalha com matrizes e `DataFrames`. Para discretizar uma série ou um vetor, usamos a classe `LabelEncoder`.

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
tennis_le = LabelEncoder()

In [ ]:
tennis_le.fit(y);

In [ ]:
yenc = tennis_le.transform(y);

Para treinar o modelo, basta usar o método `fit` sobre os exemplos de treinamento.

In [ ]:
nb.fit(Xenc, yenc)

CategoricalNB(alpha=1e-10)

Vamos testar com o exemplo do dia 15.

In [ ]:
dia15 = ['ensolarado', 'moderado', 'alta', 'forte']

In [ ]:
dia15


['ensolarado', 'moderado', 'alta', 'forte']

Para codificar esse exemplo em valores ordinais, ele precisa estar contido em uma matriz ou um *data frame*. Basta

In [ ]:
Xteste = tennis_oe.transform([dia15])
Xteste

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


array([[1., 1., 0., 0.]])

E agora usamos o método `predict()` para classificar.

In [ ]:
resultado = nb.predict(Xteste)
resultado
#array([0])

array([0])

Para nos certificarmos de que ele previu a classe como esperado, usamos o `LabelEncoder` com uma transformação inversa:

In [ ]:
tennis_le.inverse_transform(resultado)

array(['nao'], dtype=object)